In [ ]:
import torch
from os import environ
environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"
torch.backends.cudnn.allow_tf32 = True

In [ ]:
import json

# Function to get image paths for a given set of JSONs from VALID
def validLoader(filePaths):
    featureList = []
    depthList = []
    targetList = []
    jsonRoot = "VALID2020/label/"
    depthRoot = "VALID2020/depth/"
    vRoot = "VALID2020/"
    for file in filePaths:
        jsonPath = jsonRoot + file
        with open(jsonPath.rstrip(), 'r') as jsonFile:
            validx = json.load(jsonFile)
        featureList.append(vRoot + validx["file_name"])
        depthList.append(depthRoot + validx["id"] + ".png")
        targetList.append(vRoot + validx["segmentation"]["semantic_filename"])

    n = len(featureList)
    if n != len(targetList):
        print("Error: Number of feature images != number of targets.")
    elif n != len(depthList):
        print("Error: Number of feature images != number of depth images.")
    else:
        print(f"Found {n} images for dataset.")
        return n, featureList, depthList, targetList

In [ ]:
# Get list of files for segmentation training dataset
with open("VALID2020/splits/seg_train.txt", 'r') as file:
    trainFiles = file.readlines()
trainLength, segTrainFeatures, segTrainDepth, segTrainTargets = validLoader(trainFiles)

# Get list of files for segmentation testing dataset
with open("VALID2020/splits/seg_test.txt", 'r') as file:
    testFiles = file.readlines()
testLength, segTestFeatures, segTestDepth, segTestTargets = validLoader(testFiles)

In [ ]:
import json
import torch

# Extract category RGB values from dataset JSON
with open("VALID2020/categories.json", 'r') as catFile:
    catJson = json.load(catFile)
catNames = []
catColors = []
for item in catJson:
    catNames.append(item["category_name"])
    catColors.append(item["color"])

# Convert color list to tensor
catColors = torch.tensor(catColors, dtype=torch.uint8)
print(catColors.shape)

In [ ]:
import torch
nCat = len(catNames)

# Function to convert RGB tensor to one-channel category tensor
def validLabeler(label):
    # Assume label has dims [RGB, height, width]
    assert label.shape[0] == 3, "Label image input must have 3 channels"
    height = label.shape[1]
    width = label.shape[2]

    # First encode as index-hot, with a channel for each category
    output = torch.zeros(size = (nCat, height, width))
    for i in range(nCat):
        output[i] = torch.where(
            torch.eq(
                label,
                catColors[i].view(3,1,1)
            )[0],
            i,  # if same
            0   # otherwise
        )
    # Then condense to 2D, with all indices on one channel
    output = torch.sum(output, dim = 0)
    return output

In [ ]:
import torch
from torch.utils.data import Dataset
from torchvision.io import read_image, ImageReadMode
from cv2 import imread
from torchvision.transforms.functional import crop
from torchvision.transforms import ColorJitter
from torchvision.transforms.functional import resize

newSize = 256 # reduced resolution in order to reduce training time

# Create pytorch dataset for making a data loader
class Semantic(Dataset):
    def __init__(self, featurePaths, depthPaths, labelPaths):
        # each *Paths input should be a list of paths to images
        self.features = featurePaths
        self.depth = depthPaths
        self.labels = labelPaths
    
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        # Get items, preprocessing depth per VALID docs
        image = read_image(self.features[idx],
                           mode = ImageReadMode.RGB)
        depth = imread(self.depth[idx],-1) / 256
        depth = torch.from_numpy(depth)
        depth = depth.view(1, depth.shape[0], depth.shape[1])
        label = validLabeler(read_image(self.labels[idx],
                                        mode = ImageReadMode.RGB))

        # Randomly crop all three (the same)
        x = torch.rand(3)
        x[0] = (x[0] * (image.shape[-2] - newSize))*0.95  # top edge
        x[1] = (x[1] * (image.shape[-1] - newSize))*0.95  # left edge
        x[2] = (x[2]+1) * newSize    # size of crop (must be square)
        x = x.type(torch.int)

        image = crop(img = image,
                    top = x[0], left = x[1], height = x[2], width = x[2])
        depth = crop(img = depth,
                    top = x[0], left = x[1], height = x[2], width = x[2])
        label = crop(img = label,
                    top = x[0], left = x[1], height = x[2], width = x[2])
        
        # Randomly decide whether or not to flip images
        x = torch.rand(1)
        if x > 0.5:
            image = image.flip(-1)
            depth = depth.flip(-1)
            label = label.flip(-1)
        
        # Randomly jitter RGB image
        jitter = ColorJitter(brightness=0.4, contrast=0.3, saturation=0.3, hue=0.05)
        image = jitter(image)
        
        # Resize to ensure correct dimensions
        image = resize(image, newSize).view(3, newSize, newSize).type(torch.float32)
        depth = resize(depth, newSize).view(   newSize, newSize).type(torch.float32)
        label = resize(label.view(1, label.shape[0],label.shape[1]),
                        newSize).view(newSize, newSize).type(torch.long)
        return image, depth, label

In [ ]:
from torch.utils.data import DataLoader

train_batchsize = 10 # must divide evenly from/into 10
test_batchsize = 20

# Create training data loader
trainVALID = Semantic(segTrainFeatures, segTrainDepth, segTrainTargets)
trainLoader = DataLoader(
    dataset = trainVALID,
    batch_size=train_batchsize,
    shuffle=True
)

# Create testing data loader
testVALID = Semantic(segTestFeatures, segTestDepth, segTestTargets)
testLoader = DataLoader(
    dataset = testVALID,
    batch_size = test_batchsize,
    shuffle = False
)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from ShapeConv.rgbd_seg.models.utils.conv_module import build_conv_layer as dConv

# Create model class
class SaCNN(nn.Module):
    def __init__(self, mode):
        assert mode in ["ShapeConv", "Conv"], "Mode must be either ShapeConv or Conv!"
        super().__init__()
        
        # First layer, called conv1_x in Wang 2018
        self.conv1a = dConv(cfg = {"type":mode},
                            in_channels = 4,
                            out_channels = 64,
                            kernel_size = 3,
                            stride = 1,
                            padding = 1,
                            dilation = 1,
                            bias = True)

        self.conv1b = dConv(cfg = {"type":"Conv"},
                                in_channels = 64,
                                out_channels = 64,
                                kernel_size = 3,
                                stride = 1,
                                padding = 1,
                                dilation = 1)
        self.conv1c = nn.MaxPool2d(kernel_size = 3,
                                    stride = 2,
                                    padding = 1)

        # Second layer, called conv2_x in Wang 2018
        self.conv2a = dConv(cfg = {"type":mode},
                            in_channels = 64,
                            out_channels = 128,
                            kernel_size = 3,
                            stride = 1,
                            padding = 1,
                            dilation = 1)
        self.conv2b = dConv(cfg = {"type":"Conv"},
                                in_channels = 128,
                                out_channels = 128,
                                kernel_size = 3,
                                stride = 1,
                                padding = 1,
                                dilation = 1)
        
        self.conv2c = nn.MaxPool2d(kernel_size = 3,
                                    stride = 2,
                                    padding = 1)

        # Third layer, called conv3_x in Wang 2018
        self.conv3a = dConv(cfg = {"type":mode},
                            in_channels = 128,
                            out_channels = 256,
                            kernel_size = 3,
                            stride = 1,
                            padding = 1,
                            dilation = 1)
        self.conv3b = dConv(cfg = {"type":"Conv"},
                                in_channels = 256,
                                out_channels = 256,
                                kernel_size = 3,
                                stride = 1,
                                padding = 1,
                                dilation = 1)
        self.conv3c = dConv(cfg = {"type":"Conv"},
                                in_channels = 256,
                                out_channels = 256,
                                kernel_size = 3,
                                stride = 1,
                                padding = 1,
                                dilation = 1)
        self.conv3d = nn.MaxPool2d(kernel_size = 3,
                                    stride = 2,
                                    padding = 1)

        # Fourth layer, called conv4_x in Wang 2018
        self.conv4a = dConv(cfg = {"type":mode},
                            in_channels = 256,
                            out_channels = 512,
                            kernel_size = 3,
                            stride = 1,
                            padding = 1,
                            dilation = 1)
        self.conv4b = dConv(cfg = {"type":"Conv"},
                                in_channels = 512,
                                out_channels = 512,
                                kernel_size = 3,
                                stride = 1,
                                padding = 1,
                                dilation = 1)
        self.conv4c = dConv(cfg = {"type":"Conv"},
                                in_channels = 512,
                                out_channels = 512,
                                kernel_size = 3,
                                stride = 1,
                                padding = 1,
                                dilation = 1)
        self.conv4d = nn.MaxPool2d(kernel_size = 3,
                                    stride = 2,
                                    padding = 1)

        # Fifth layer, called conv5_x in Wang 2018
        self.conv5a = dConv(cfg = {"type":mode},
                            in_channels = 512,
                            out_channels = 512,
                            kernel_size = 3,
                            stride = 1,
                            padding = 1,
                            dilation = 2)
        self.conv5b = dConv(cfg = {"type":"Conv"},
                                in_channels = 512,
                                out_channels = 512,
                                kernel_size = 3,
                                stride = 1,
                                padding = 1,
                                dilation = 2)
        self.conv5c = dConv(cfg = {"type":"Conv"},
                                in_channels = 512,
                                out_channels = 512,
                                kernel_size = 3,
                                stride = 1,
                                padding = 1,
                                dilation = 2)
        self.conv5d = nn.AvgPool2d(kernel_size = 3,
                                    stride = 1,
                                    padding = 1)

        # Final layers, called conv6 & conv7 in Wang 2018
        self.conv6 = dConv(
            cfg = {"type":mode},
            in_channels = 512,
            out_channels = 1024,
            kernel_size = 3,
            stride = 1,
            padding = 12,
            dilation = 12
        )

        self.conv7 = dConv(
            cfg = {"type":"Conv"},                                 
            in_channels = 1024,
            out_channels = 1024,
            kernel_size = 1,
            stride = 1,
            padding = 0
        )

        # Scale predictions up to full resolution from latent space
        self.convFinal = nn.Sequential(
            nn.ConvTranspose2d(
                in_channels = 2048, # double channels via globalpool + concat
                out_channels = 1024,
                kernel_size = 4,
                stride = 4,        # expand from 10x10 to 32x32
                padding = 4
            ),
            nn.ReLU(),
            nn.ConvTranspose2d(
                in_channels = 1024,
                out_channels = 512,
                kernel_size = 4,
                stride = 4,        # expand from 32x32 to 128x128
                padding = 0
            ),
            nn.ReLU(),
            nn.ConvTranspose2d(
                in_channels = 512,
                out_channels = nCat,
                kernel_size = 2,
                stride = 2,        # expand from 128x128 to 256x256
                padding = 0
            )
        )
    
    def forward(self, x, d):
        # Channel-wise concatenation converts x from RGB to RGB-D
        x = torch.cat((x, d.view(d.shape[0], 1, d.shape[1], d.shape[2])), 1)

        # conv1_x
        x = F.relu(self.conv1a(x))
        x = F.relu(self.conv1b(x))
        x = self.conv1c(x)

        # conv2_x
        x = F.relu(self.conv2a(x))
        x = F.relu(self.conv2b(x))
        x = self.conv2c(x)

        # conv3_x
        x = F.relu(self.conv3a(x))
        x = F.relu(self.conv3b(x))
        x = F.relu(self.conv3c(x))
        x = self.conv3d(x)

        # conv4_x
        x = F.relu(self.conv4a(x))
        x = F.relu(self.conv4b(x))
        x = F.relu(self.conv4c(x))
        x = self.conv4d(x)

        # conv5_x
        x = F.relu(self.conv5a(x))
        x = F.relu(self.conv5b(x))
        x = F.relu(self.conv5c(x))
        x = self.conv5d(x)

        # conv6 & conv7
        x = F.relu(self.conv6(x))
        x = F.relu(self.conv7(x))

        # globablpool + concat
        globalpool = F.adaptive_avg_pool2d(input = x, output_size = (1,1))
        globalpool = globalpool.repeat(1,1,x.shape[2], x.shape[3])
        x = torch.cat((x, globalpool), 1)
        x = self.convFinal(x)
        return x

In [ ]:
import torch

# Enable GPU acceleration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
import torch
import torch.nn.functional as F

# Create training function
def train(epoch, model, device, optimizer, data = trainLoader):
    # Prepare model and inputs
    torch.cuda.empty_cache()
    model.train()
    for batch_idx, (images, depths, labels) in enumerate(data):
        images = images.to(device)
        depths = depths.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()

        # Calculate and record the output and loss
        output = model(images, depths)
        loss = F.cross_entropy(output, labels)
        loss.backward()
        optimizer.step()

        # Periodically update with progress
        print(f"\rEpoch {epoch}: Training {batch_idx*len(images)}/{len(data.dataset)} "
              f"(Loss: {loss.item():8.6})", end="")
        if batch_idx % int(10/train_batchsize) == 0:
            for g in optimizer.param_groups: g['lr'] *= pow(1 - (batch_idx/trainLength), 0.9)
    print(f"\rEpoch {epoch}: Training {len(data.dataset)}/{len(data.dataset)} "
          f"(Loss: {loss.item():8.6})")

In [ ]:
import torch
import torch.nn.functional as F

# Create testing function
def test(epoch, model, device, data = testLoader):
    # Prepare model and inputs
    torch.cuda.empty_cache()
    model.eval()
    test_loss = []
    correct = []
    with torch.no_grad():
        for batch_idx, (images, depths, labels) in enumerate(data):
            print(f"\rEpoch {epoch}: "
                  f"Testing {batch_idx*len(images)}/{len(data.dataset)}...", end="")
            images = images.to(device)
            depths = depths.to(device)
            labels = labels.to(device)

            # Calculate and record output and loss
            output = model(images, depths)
            test_loss.append(F.cross_entropy(output, labels).item())

            n = torch.prod(torch.tensor(labels.shape))
            prediction = output.data.max(1, keepdim = True).indices
            correct.append(prediction.eq(labels.data.view_as(prediction)).sum() / n)
        print(f"\rEpoch {epoch}: Testing {len(data.dataset)}/{len(data.dataset)}...")
    
    # Report results
    test_loss = torch.mean(torch.tensor(test_loss))
    accuracy = 100 * torch.mean(torch.tensor(correct))
    print(f"Test result, epoch {epoch}: Avg loss {test_loss:6.4}, Accuracy {accuracy:6.4}%")

In [ ]:
# Function to convert a time difference (seconds) to a human-readable string
def dTime(inTime):
    if inTime >= (60*60):    # Hours
        hour = f"{(inTime // (60*60)):2.0f}"
        inTime = inTime % (60*60)
        mins = f"{(inTime // 60):02.0f}"
        secs = f"{(inTime %  60):02.0f}"
        output = hour + ":" + mins + ":" + secs
    elif inTime >= 60*2:     # Minutes
        mins = f"{(inTime // 60):02.0f}"
        secs = f"{(inTime %  60):02.0f}"
        output = mins + ":" + secs
    else:                   # Seconds
        secs = round(inTime, 1)
        output = f"{secs} sec"
    return output

In [ ]:
import time

# Function to train & test several models with same epochs, optimizer, etc
def trainTest(max_epoch, models, device, optim, lr0):
    # Max_epoch is the number of epochs to train & test for each model
    # Models is a list of models to train & test
    # Optimizer is the optimizer to use while training each model
    nModels = len(models)
    start = time.time()

    for i in range(nModels):
        print(f" ======== Model {i+1} of {nModels} ========")
        models[i] = models[i].to(device)
        optimizer = optim(models[i].parameters(), lr=lr0)
        
        mAfter = nModels - (i+1) # Number of models after this one
        mStart = time.time()

        for epoch in range(1, max_epoch+1):
            if epoch != 1:       # Print a timing report after all but last epoch
                elapsed = time.time() - mStart
                modelEst = (elapsed * max_epoch / (epoch-1))
                remain = (modelEst - elapsed) + (mAfter * modelEst)
                print(f"   {dTime(elapsed)} elapsed, "
                      f"estimate {dTime(remain)} remaining")
            
            # Train and test for this epoch
            train(epoch, models[i], device, optimizer)
            test(epoch, models[i], device)
        
        # Remove model from GPU when finished
        models[i] = models[i].to('cpu')

        # Estimate remaining time
        elapsed = time.time() - start
        remain = (elapsed / (i+1)) * mAfter
        if i != (nModels-1): # don't print an estimate when finished last model
            print(f"   {dTime(elapsed)} elapsed, estimate {dTime(remain)} remaining")
    print(f" ==== Completed {nModels} models in {dTime(time.time() - start)} ====")

From a pilot study, the model converges after about 5-8 epochs:
```
Test result, epoch 1: Avg loss  2.164, Accuracy  22.13%
10:30 elapsed, estimate 1:34:28 remaining

Test result, epoch 2: Avg loss  2.141, Accuracy  21.86%
20:59 elapsed, estimate 1:23:55 remaining

Test result, epoch 3: Avg loss  2.135, Accuracy  22.24%
31:27 elapsed, estimate 1:13:22 remaining

Test result, epoch 4: Avg loss  2.089, Accuracy  24.32%
42:5 elapsed, estimate 1:03:07 remaining

Test result, epoch 5: Avg loss  2.039, Accuracy  25.21%
52:34 elapsed, estimate 52:34 remaining

Test result, epoch 6: Avg loss  2.002, Accuracy  27.88%
1:3:8 elapsed, estimate 42:05 remaining

Test result, epoch 7: Avg loss  1.995, Accuracy  26.53%
1:13:38 elapsed, estimate 31:34 remaining

Test result, epoch 8: Avg loss  1.978, Accuracy  26.78%
1:24:12 elapsed, estimate 21:03 remaining

Test result, epoch 9: Avg loss  1.974, Accuracy  25.94%
Total time 1:34:43 elapsed
```

In [ ]:
import torch.optim as optim

modes = ["ShapeConv", "Conv"]
depthModel = SaCNN(modes[0])
controlModel = SaCNN(modes[1])
models = [depthModel, controlModel]

trainTest(7, models, device, optim.Adam, lr0=0.0001)


In [ ]:
# Extract a few images from test dataset to make figures
images, depths, labels = next(iter(testLoader))
images = images.to(device)
depths = depths.to(device)
labels = labels.to(device)

In [ ]:
# Get predictions for each model
outputs = []
predictions = []
for i in range(len(models)):
    models[i].to(device)
    outputs.append(models[i](images, depths))
    predictions.append(outputs[i].data.max(1, keepdim = True).indices)
    models[i].to('cpu')

In [ ]:
# A function to take a category-hot tensor and turn it into an RGB tensor
def validUnlabeler(input, catColors = catColors):
    assert input.ndim == 3, "Input tensor must be batch x height x width"
    assert input.max() <= len(catColors), "Input has category indices out of range!"
    assert input.min() >= 0, "Input cannot have negative values!"

    # Output dimensions: batch x channel x height x width
    b, h, w = input.shape[0], input.shape[1], input.shape[2]
    R = torch.zeros(size = (b, nCat, h, w))
    G = torch.zeros(size = (b, nCat, h, w))
    B = torch.zeros(size = (b, nCat, h, w))

    # First make 3 tensors with nCat channels
    for b in range(input.shape[0]):
        for i in range(nCat):
            R[b,i] = torch.where(
                torch.eq(input[b], i),
                catColors[i,0], # if in category i
                0)              # zero otherwise
            G[b,i] = torch.where(
                torch.eq(input[b], i),
                catColors[i,1],
                0)
            B[b,i] = torch.where(
                torch.eq(input[b], i),
                catColors[i,2],
                0)
    
    # Collapse all 3 tensors along nCat dimension
    R = R.sum(1, keepdim=True)
    G = G.sum(1, keepdim=True)
    B = B.sum(1, keepdim=True)

    # Concatenate all 3 tensors to make 1 tensor with 3 channels
    output = torch.cat((R,G,B),1).type(torch.uint8)
    return(output)

In [ ]:
# Convert tensors to RGB and put on CPU
images = images.to('cpu').type(torch.uint8)
depths = depths.to('cpu').type(torch.uint8)
labelsRGB = validUnlabeler(labels.to('cpu'))

predictionRGB = []
for i in range(len(models)):
    predictionRGB.append(
        validUnlabeler(
            predictions[i].to('cpu').view(predictions[i].shape[0],
                                          predictions[i].shape[-2],
                                          -1)
        )
    )                  

In [ ]:
import matplotlib.pyplot as plt

# Make plots of tensors
nRows = min(images.shape[0], 5)
nCols = 3 + len(predictionRGB)
fig, axs = plt.subplots(nrows=nRows, ncols=nCols, figsize=(8, 10), dpi=150)

for b in range(nRows):
    # First column: RGB Image
    ax = axs[b, 0]
    if b == 0: ax.title.set_text("RGB Image")
    ax.imshow(images[b].permute(1,2,0).numpy())
    ax.axis('off')

    # Second column: Depth Image
    ax = axs[b, 1]
    if b == 0: ax.title.set_text("Depth Data")
    ax.imshow(depths[b].numpy(), cmap='gray')
    ax.axis('off')

    for i in range(len(predictionRGB)):
        # Third column: Modeled Segmentation
        ax = axs[b, 2+i]
        if b == 0: ax.title.set_text(modes[i])
        ax.imshow(predictionRGB[i][b].permute(1,2,0).numpy())
        ax.axis('off')

    # Last Column: Ground Truth
    ax = axs[b, -1]
    if b == 0: ax.title.set_text("Ground Truth")
    ax.imshow(labelsRGB[b].permute(1,2,0).numpy())
    ax.axis('off')